In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [3]:
import openfermion as of
import openfermionpyscf as ofpyscf

geometry=[
    ('H', (0.0,0.0,0.0)),
    ('H', (2.45366053071732,0.0,0.0)),
    ('H', (2.45366053071732,2.45366053071732,0.0)),
    ('H', (0.0,2.45366053071732,0.0))
     ]    
    
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
#molecule.load()

# Run pyscf.
molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=1,run_ccsd=1,run_fci=1)

n_qubits    = 2*molecule.n_orbitals
n_electrons = molecule.n_electrons

ham_fermionic = of.get_fermion_operator(molecule.get_molecular_hamiltonian())

ccsd_single_amps = molecule_data.ccsd_single_amps
ccsd_double_amps = molecule_data.ccsd_double_amps

In [4]:
H_mat = get_sparse_operator(ham_fermionic)

In [15]:
hf_comp_basis_state = np.zeros(molecule.n_qubits, dtype=int)
hf_comp_basis_state[:molecule.n_electrons] = 1

binary_int_list = 1 << np.arange(molecule.n_qubits)[::-1]
hf_ket = np.zeros(2 ** molecule.n_qubits, dtype=int)
hf_ket[hf_comp_basis_state @ binary_int_list] = 1

In [6]:
double_amplitudes_list=[]
double_amplitudes = ccsd_double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
double_amplitudes_list

[[[4, 0, 5, 1], -0.5456322187873451],
 [[4, 0, 6, 2], -1.1613365888368996e-06],
 [[4, 0, 7, 3], 5.991199944039062e-07],
 [[4, 1, 5, 0], 0.5456322187873451],
 [[4, 1, 7, 2], -1.7604565832408058e-06],
 [[4, 2, 5, 3], 0.03662801351783161],
 [[4, 2, 6, 0], 1.1613365888368996e-06],
 [[4, 2, 7, 1], 1.7604565832408058e-06],
 [[4, 3, 5, 2], -0.03662801351783161],
 [[4, 3, 7, 0], -5.991199944039062e-07],
 [[5, 0, 4, 1], 0.5456322187873451],
 [[5, 0, 6, 3], -1.7604565832408058e-06],
 [[5, 1, 4, 0], -0.5456322187873451],
 [[5, 1, 6, 2], 5.991199944039062e-07],
 [[5, 1, 7, 3], -1.1613365888368996e-06],
 [[5, 2, 4, 3], -0.03662801351783161],
 [[5, 2, 6, 1], -5.991199944039062e-07],
 [[5, 3, 4, 2], 0.03662801351783161],
 [[5, 3, 6, 0], 1.7604565832408058e-06],
 [[5, 3, 7, 1], 1.1613365888368996e-06],
 [[6, 0, 4, 2], 1.1613365888368996e-06],
 [[6, 0, 5, 3], 1.7604565832408058e-06],
 [[6, 0, 7, 1], 0.0006616364017180414],
 [[6, 1, 5, 2], -5.991199944039062e-07],
 [[6, 1, 7, 0], -0.0006616364017180414]

In [7]:
single_amplitudes_list = []
for i, j in zip(*ccsd_single_amps.nonzero()):
    single_amplitudes_list.append([[i, j], ccsd_single_amps[i, j]])

single_amplitudes_list

[[[4, 0], 3.7372991954509263e-14],
 [[4, 2], 3.012770106836085e-06],
 [[5, 1], 3.7372991954509263e-14],
 [[5, 3], 3.012770106836085e-06],
 [[6, 0], -0.44350284282277974],
 [[6, 2], 7.37425160928958e-14],
 [[7, 1], -0.44350284282277974],
 [[7, 3], 7.37425160928958e-14]]

In [8]:
generator_t2 = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator_t2 += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [9]:
generator_t1 = FermionOperator()
for (i, j), t_ij in single_amplitudes_list:
    i, j = int(i), int(j)
    generator_t1 += FermionOperator(((i, 1), (j, 0)), t_ij)

In [12]:
T = generator_t1 + 0.5*generator_t2

ccsd_ansatz = expm(get_sparse_operator(T, n_qubits=molecule.n_qubits))

In [16]:
ccsd_ket =  ccsd_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ ccsd_ket

(-1.8261576342039731+0j)

In [19]:
molecule.ccsd_energy

-1.8261576342041543

In [31]:
hf_ket.conj().T @ ccsd_ket

(1+0j)

In [21]:
for ind, amp in enumerate(ccsd_ket):
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=molecule.n_qubits)
        print(f'{amp}|{binary_string}>')

(1.2183051954059552+0j)|00001111>
(1.995064311144928e-06+0j)|00011011>
(-1.995064311144928e-06+0j)|00100111>
(0.19801804439532333+0j)|00110011>
(-1.0912644375746903+0j)|00111100>
(0.03248925623063924+0j)|01001110>
(9.86501070520506e-07+0j)|01011010>
(-2.1847410593283184e-06+0j)|01100110>
(1.1982399888078124e-06+0j)|01101001>
(0.4435028428227797+0j)|01110010>
(-0.032489256230639245+0j)|10001101>
(1.1982399888078124e-06+0j)|10010110>
(-2.1847410593283184e-06+0j)|10011001>
(9.86501070520506e-07+0j)|10100101>
(-0.44350284282277974+0j)|10110001>
(-1.1031232565978135+0j)|11000011>
(0.07325602704473999+0j)|11001100>
(-3.012770106836085e-06+0j)|11011000>
(3.012770106836085e-06+0j)|11100100>
(1+0j)|11110000>


In [28]:
no_hf_ket = ccsd_ket - hf_ket
max_amp_binary_string = np.binary_repr(np.argsort(np.abs(no_hf_ket))[::-1][0], width=molecule.n_qubits)
print(max_amp_binary_string)

00001111


look at states with common sector (aka assign eigval of stabilizers in the same way)

Gives a way of searching over sectors in a reduced approach.

In [37]:
# go by size
no_hf_ket = ccsd_ket - hf_ket
for ind in (np.argsort(np.abs(no_hf_ket))[::-1]):
    amp = no_hf_ket[ind]
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=molecule.n_qubits)
        print(f'{amp}|{binary_string}>')

(1.2183051954059552+0j)|00001111>
(-1.1031232565978135+0j)|11000011>
(-1.0912644375746903+0j)|00111100>
(-0.44350284282277974+0j)|10110001>
(0.4435028428227797+0j)|01110010>
(0.19801804439532333+0j)|00110011>
(0.07325602704473999+0j)|11001100>
(-0.032489256230639245+0j)|10001101>
(0.03248925623063924+0j)|01001110>
(3.012770106836085e-06+0j)|11100100>
(-3.012770106836085e-06+0j)|11011000>
(-2.1847410593283184e-06+0j)|10011001>
(-2.1847410593283184e-06+0j)|01100110>
(-1.995064311144928e-06+0j)|00100111>
(1.995064311144928e-06+0j)|00011011>
(1.1982399888078124e-06+0j)|01101001>
(1.1982399888078124e-06+0j)|10010110>
(9.86501070520506e-07+0j)|01011010>
(9.86501070520506e-07+0j)|10100101>


In [46]:
ccsd_ket =  ccsd_ansatz @ hf_ket
norm_ccsd = ccsd_ket/np.linalg.norm(ccsd_ket)
norm_ccsd.conj().T @ H_mat @ norm_ccsd

(-1.741903772141814+0j)

In [47]:
molecule.ccsd_energy

-1.8261576342041543